<a href="https://colab.research.google.com/github/serenedsouza/ML-Assignments/blob/main/LLAMA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Llama 2](https://llama.meta.com/llama2) and Model Fine-Tuning

**Llama 2** is a collection of second-generation open-source Large Language Models (LLMs) from Meta, designed to handle a wide range of natural language processing tasks. These models range in scale from `7 billion to 70 billion parameters`.

**Llama-2-Chat**, optimized for dialogue, has shown similar performance to popular closed-source models like ChatGPT and PaLM.

**Fine-tuning** in machine learning involves adjusting the weights and parameters of a pre-trained model on new data to improve its performance on a specific task. It includes training the model on a new dataset specific to the task at hand, while updating the model's weights to adapt to the new data.

<div align="center">
<img src = "https://images.datacamp.com/image/upload/v1697724450/Fine_Tune_L_La_MA_2_cc6aa0e4ad.png">
</div>

In [1]:
# %pip install accelerate peft bitsandbytes transformers trl
!pip install -U datasets trl accelerate peft bitsandbytes transformers trl huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Fo

## Importing Necessary Libraries

In [2]:
import os
import pandas as pd
import torch

from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTConfig, SFTTrainer
from huggingface_hub import login

print(torch.__version__)

# Setting up device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

2.5.0+cu121
cuda


In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Model Configuration
Using NousResearch’s `Llama-2-7b-chat-hf` as our base model. It is the same as the original Meta’s official `Llama-2 model` from Hugging Face but easily accessible.

### [See Guanaco Dataset](https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k)

In [4]:
# Model from Hugging Face hub with 7 billion parameters
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama2-7B-finetuned-chat-guanaco"

## Loading dataset, model, and tokenizer

In [5]:
dataset = load_dataset(guanaco_dataset, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

(…)-00000-of-00001-9ad84bb9cf65a42f.parquet:   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## 4-bit Quantization Configuration
4-bit Quantization via QLoRA allows efficient finetuning of huge LLM models on consumer hardware while retaining high performance. This dramatically improves accessibility and usability for real-world applications.

QLoRA quantizes a pre-trained language model to 4 bits and freezes the parameters. A small number of trainable Low-Rank Adapter layers are then added to the model.

During fine-tuning, gradients are backpropagated through the frozen 4-bit quantized model into only the Low-Rank Adapter layers. So, the entire pretrained model remains fixed at 4 bits while only the adapters are updated. Also, the 4-bit quantization does not hurt model performance.

<img src = "https://images.datacamp.com/image/upload/v1697713094/image7_3e12912d0d.png">

### [Paper on QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)

In [6]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",             # Taking nf4 4bit quantization
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

## Loading `Llama 2 model`

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

## Load the Tokenizers

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## PEFT Parameters
Traditional fine-tuning of pre-trained language models (PLMs) requires updating all of the model's parameters, which is computationally expensive and requires massive amounts of data.

Parameter-Efficient Fine-Tuning (PEFT) works by only updating a small subset of the model's parameters, making it much more efficient. Learn about parameters by reading the [PEFT official documentation](https://huggingface.co/docs/peft/conceptual_guides/lora).

In [9]:
peft_params = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = "none",
    task_type = "CAUSAL_LM",)

## Training Hyperparameters

In [10]:
training_params = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=3,              # Epochs to train
    per_device_train_batch_size=8,   # Batch_size for train

    gradient_accumulation_steps=1,   # Aggressively accumulate gradients to compensate for low batch size
    optim="adamw_torch",             # Efficient optimizer for LLMs
    save_steps=50,                   # Adjust saving frequency based on training duration
    logging_steps=25,                # Adjust logging frequency based on your preference
    learning_rate=2e-5,              # Start with very low learning rate to mitigate instability
    weight_decay=0.01,               # Regularization to prevent overfitting

    fp16=True,                       # Enable mixed precision for memory savings
    bf16=False,                      # T4 doesn't support bfloat16
    max_grad_norm=0.3,               # Adjust gradient norm as needed
    max_steps=-1,                    # Train for all epochs by default
    warmup_ratio=0.03,               # Adjust warmup ratio based on learning rate and dataset size
    group_by_length=True,            # Improve efficiency for long sequences
    lr_scheduler_type="constant",    # Use warmup followed by constant learning rate
    report_to="tensorboard",         # Track training progress with TensorBoard

    # NOTE: Additional memory-specific optimizations:

    # max_train_steps = 1000,        # Set a maximum number of training steps to limit total memory usage
    # sharded_ddp = True,            # Enable DistributedDataParallel sharding if multiple GPUs are available
    gradient_checkpointing = True,   # Recompute intermediate activations for memory savings
    fp16_full_eval = True,           # Use mixed precision during evaluation as well
    dataloader_pin_memory = False,   # Disable data pinning to avoid potential memory overhead
    local_rank = -1,                 # Disable automatic distributed training (if only 1 GPU)
    # skip_memory_check=True,        # Temporarily skip memory checks, but monitor closely

    push_to_hub=True,                # Save checkpoint in Hugging Face Hub
)

## Model fine-tuning
Supervised fine-tuning (SFT) is a key step in reinforcement learning from human feedback (RLHF). The TRL library from HuggingFace provides an easy-to-use API to create SFT models and train them on your dataset with just a few lines of code. It comes with tools to train language models using reinforcement learning, starting with supervised fine-tuning, then reward modeling, and finally proximal policy optimization (PPO).

Provide SFT Trainer the model, dataset, Lora configuration, tokenizer, and training parameters.

In [11]:
sft_config = SFTConfig(
    output_dir=new_model,
    dataset_text_field="text",
    max_seq_length=512,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=sft_config.max_seq_length,
    dataset_text_field=sft_config.dataset_text_field,
    peft_config=peft_params,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.817400
50,2.041400
75,1.817500
100,1.733300
125,1.706300
150,1.407800
175,1.497700
200,1.438000
225,1.491500
250,1.427100


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

Step,Training Loss
25,1.817400
50,2.041400
75,1.817500
100,1.733300
125,1.706300
150,1.407800
175,1.497700
200,1.438000
225,1.491500
250,1.427100


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=375, training_loss=1.5404170939127604, metrics={'train_runtime': 3667.1573, 'train_samples_per_second': 0.818, 'train_steps_per_second': 0.102, 'total_flos': 4.219945382805504e+16, 'train_loss': 1.5404170939127604, 'epoch': 3.0})

In [13]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/Seerene/llama2-7B-finetuned-chat-guanaco/commit/a8890d9d163484b08dbae9bb6f8d4bbd769854cd', commit_message='End of training', commit_description='', oid='a8890d9d163484b08dbae9bb6f8d4bbd769854cd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Seerene/llama2-7B-finetuned-chat-guanaco', endpoint='https://huggingface.co', repo_type='model', repo_id='Seerene/llama2-7B-finetuned-chat-guanaco'), pr_revision=None, pr_num=None)

In [14]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('llama2-7B-finetuned-chat-guanaco/tokenizer_config.json',
 'llama2-7B-finetuned-chat-guanaco/special_tokens_map.json',
 'llama2-7B-finetuned-chat-guanaco/tokenizer.model',
 'llama2-7B-finetuned-chat-guanaco/added_tokens.json',
 'llama2-7B-finetuned-chat-guanaco/tokenizer.json')

In [ ]:
# from tensorboard import notebook
# log_dir = "results/runs"
# notebook.start("--logdir {} --port 4000".format(log_dir))

## Testing Text Generation

In [17]:
import logging

# Set logging verbosity
logging.basicConfig(level=logging.CRITICAL)

config = {
    "task": "text-generation",
    "model": model,
    "tokenizer": tokenizer,
    "max_length": 500,
    "config": {
        "language": "en"
    }
}

In [18]:
prompt = "Define github?"

pipe = pipeline(**config)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Define github?
 everybody uses it. but what is it?

Github is a platform for developers to share their projects with the world. It allows users to store their code online, collaborate with others on projects, and track changes to their code over time.

Github is similar to a social network for developers, where they can share their projects, collaborate with other developers, and get feedback on their work.

Github is also a place where developers can store their code online, so they can access it from anywhere, and collaborate with others on projects.

Github is a powerful tool for developers, allowing them to share their work with others, collaborate on projects, and track changes to their code over time.

Github is a great tool for developers, allowing them to share their work with others, collaborate on projects, and track changes to their code over time.

Github is a platform for developers to share their projects with the world, collaborate with others on projects, and track chan

In [19]:
input = "Types of Cyber Threats"
index = 1

# Here are the names of different types of Graphviz diagrams:
# Flowchart
# Hierarchical Diagram
# Network Diagram
# State Machine Diagram
# Mind Map
# Entity-Relationship Diagram (ERD)
# UML Class Diagram

a = ["Flowchart",
"Hierarchical Diagram",
"Network Diagram",
"State Machine Diagram",
"Mind Map",
"Entity-Relationship Diagram (ERD)",
"UML Class Diagram"
]
prompt = f'Generate a  Graphviz code for ${a[index]} on " ${input} " and provide only the Graphviz code between "digraph G {" and "}" without any other text or explanation. wrap the code under "```" at start and end'

pipe = pipeline(**config)
result = pipe(f"{prompt}")
with open("cyber_threats_dot_code.txt", "w") as f:
    f.write(result[0]['generated_text'])

print("Dot code saved to cyber_threats_dot_code.txt")
print(result[0]['generated_text'])

Dot code saved to cyber_threats_dot_code.txt
Generate a  Graphviz code for $Hierarchical Diagram on " $Types of Cyber Threats " and provide only the Graphviz code between "digraph G  and " without any other text or explanation. wrap the code under "```" at start and end of the code.

```
digraph G {
    rankdir=TB
    node [shape=box]
    subgraph cluster {
        label="Types of Cyber Threats"
        Types[shape=ellipse]
        Types[label="Malware"]
        Types[label="Phishing"]
        Types[label="Ransomware"]
        Types[label="Distributed Denial of Service"]
        Types[label="SQL Injection"]
        Types[label="Zero-Day Exploits"]
        Types[label="Social Engineering"]
        Types[label="Physical Attacks"]
    }
}
```

```
digraph G {
    rankdir=TB
    node [shape=box]
    subgraph cluster {
        label="Types of Cyber Threats"
        Types[shape=ellipse]
        Types[label="Malware"]
        Types[label="Phishing"]
        Types[label="Ransomware"]
        T

In [20]:
!pip install graphviz

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# code_block = result[0]['generated_text']
# print("whole code")
# print(code_block)
# # Find the start and end indices of the code block

# start_index = code_block.index("```")+3   # +3 to skip past the '''
# end_index = code_block.rindex("```")  # Find the last occurrence of '''

# # Extract the substring
# substring = code_block[start_index:end_index].strip()
# print("below.........................")
# # Display the extracted substring
# print(substring)
code_block = result[0]['generated_text']
print("whole code")
print(code_block)

# Find all occurrences of the code blocks
code_blocks = code_block.split("```")

# Check if there are at least three parts (the first and last are usually empty if there are 3 backticks)
if len(code_blocks) > 2:
    # The last code block will be the second last item in the split list
    last_code_block = code_blocks[-2].strip()
    print("below.........................")
    # Display the extracted last code block
    print(last_code_block)
else:
    print("No code block found.")

whole code
Generate a  Graphviz code for $Hierarchical Diagram on " $Types of Cyber Threats " and provide only the Graphviz code between "digraph G  and " without any other text or explanation. wrap the code under "```" at start and end of the code.

```
digraph G {
    rankdir=TB
    node [shape=box]
    subgraph cluster {
        label="Types of Cyber Threats"
        Types[shape=ellipse]
        Types[label="Malware"]
        Types[label="Phishing"]
        Types[label="Ransomware"]
        Types[label="Distributed Denial of Service"]
        Types[label="SQL Injection"]
        Types[label="Zero-Day Exploits"]
        Types[label="Social Engineering"]
        Types[label="Physical Attacks"]
    }
}
```

```
digraph G {
    rankdir=TB
    node [shape=box]
    subgraph cluster {
        label="Types of Cyber Threats"
        Types[shape=ellipse]
        Types[label="Malware"]
        Types[label="Phishing"]
        Types[label="Ransomware"]
        Types[label="Distributed Denial of 

In [32]:
import graphviz
from IPython.display import Image

# Define your Graphviz code
graphviz_code = last_code_block

# Create a graph from the code
graph = graphviz.Source(graphviz_code)
output_path = "cyber_security_flowchart"  # Save path for Colab
graph.render(output_path, format='png')

# Display the image
display(Image(filename=f"{output_path}.png"))


FileNotFoundError: [Errno 2] No such file or directory: 'cyber_security_flowchart.png'

In [33]:
digraph G {
    rankdir=TB
    node [shape=box]
    subgraph cluster {
        label="Types of Cyber Threats"
        Types[shape=ellipse]
        Types[label="Malware"]
        Types[label="Phishing"]
        Types[label="Ransomware"]
        Types[label="Distributed Denial of Service"]
        Types[label="SQL Injection"]
        Types[label="Zero-Day Exploits"]
        Types[label="Social Engineering"]
        Types[label="Physical Attacks"]
    }
}

SyntaxError: invalid syntax (<ipython-input-33-cf8fa226d781>, line 1)